# Week 1 Quiz

In [4]:
import pandas as pd
import numpy as np
import edhec_risk_kit as erk
import scipy.stats
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
returns = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                     header=0, index_col=0, parse_dates=True,
                     na_values=-99.99)
returns.head()

,<= 0,Lo 30,Med 40,Hi 30,Lo 20,Qnt 2,Qnt 3,Qnt 4,Hi 20,Lo 10,Dec 2,Dec 3,Dec 4,Dec 5,Dec 6,Dec 7,Dec 8,Dec 9,Hi 10
192607,NaN,-0.43,1.52,2.68,-0.57,0.59,1.60,1.47,3.33,-1.45,0.29,-0.15,1.33,1.24,1.98,1.55,1.38,3.38,3.29
192608,NaN,3.90,3.04,2.09,3.84,3.59,3.71,1.61,2.33,5.12,2.59,4.03,3.15,2.72,4.72,1.60,1.63,0.98,3.70
192609,NaN,-1.08,-0.54,0.16,-0.48,-1.40,0.00,-0.50,-0.09,0.93,-1.87,-2.27,-0.53,0.07,-0.07,-1.64,0.64,-0.86,0.67
192610,NaN,-3.32,-3.52,-3.06,-3.29,-4.10,-2.89,-3.36,-2.95,-4.84,-1.77,-3.36,-4.83,-2.98,-2.80,-3.45,-3.27,-3.47,-2.43
192611,NaN,-0.46,3.82,3.09,-0.55,2.18,3.41,3.39,3.16,-0.78,-0.32,-0.29,4.65,3.24,3.57,3.82,2.95,3.61,2.70


In [17]:
columns = ['Lo 20', 'Hi 20']
returns = returns[columns]
returns.head()

,Lo 20,Hi 20
192607,-0.57,3.33
192608,3.84,2.33
192609,-0.48,-0.09
192610,-3.29,-2.95
192611,-0.55,3.16


In [18]:
returns = returns/100
returns.head()

,Lo 20,Hi 20
192607,-0.0057,0.0333
192608,0.0384,0.0233
192609,-0.0048,-0.0009
192610,-0.0329,-0.0295
192611,-0.0055,0.0316


In [19]:
n_months = returns.shape[0]
n_months

1110

In [20]:
return_per_month = (returns+1).prod()**(1/n_months)-1
return_per_month

Lo 20    0.011860
Hi 20    0.007859
dtype: float64

In [21]:
annualized_return = (return_per_month+1)**12-1
annualized_return 

Lo 20    0.151977
Hi 20    0.098490
dtype: float64

In [22]:
annualized_vol = returns.std()*np.sqrt(12)
annualized_vol

Lo 20    0.336701
Hi 20    0.195116
dtype: float64

In [24]:
me_m = pd.read_csv('data/Portfolios_Formed_on_ME_monthly_EW.csv',
                  header=0, index_col=0, parse_dates=True, na_values=-99.99)
rets = me_m[['Lo 20', 'Hi 20']]
rets = rets/100

In [25]:
rets.head()

,Lo 20,Hi 20
192607,-0.0057,0.0333
192608,0.0384,0.0233
192609,-0.0048,-0.0009
192610,-0.0329,-0.0295
192611,-0.0055,0.0316


In [26]:
rets.index = pd.to_datetime(rets.index, format="%Y%m")
rets.head()

,Lo 20,Hi 20
1926-07-01,-0.0057,0.0333
1926-08-01,0.0384,0.0233
1926-09-01,-0.0048,-0.0009
1926-10-01,-0.0329,-0.0295
1926-11-01,-0.0055,0.0316


In [27]:
rets.index = rets.index.to_period('M')

In [31]:
rets.loc["1999":"2015"]

,Lo 20,Hi 20
1999-01,0.1114,0.0153
1999-02,-0.0399,-0.0172
1999-03,-0.0308,0.0368
1999-04,0.0824,0.0679
1999-05,0.0435,-0.0048
...,...,...
2015-08,-0.0347,-0.0559
2015-09,-0.0610,-0.0333
2015-10,0.0385,0.0710
2015-11,0.0165,0.0015


In [33]:
n_months = rets.loc["1999":"2015"].shape[0]
n_months

204

In [34]:
ret_per_month = (rets.loc["1999":"2015"]+1).prod()**(1/n_months)-1
ret_per_month

Lo 20    0.009069
Hi 20    0.005094
dtype: float64

In [35]:
annualized_ret = (ret_per_month+1)**12-1
annualized_ret

Lo 20    0.114426
Hi 20    0.062865
dtype: float64

In [36]:
annualized_vol = rets.loc["1999":"2015"].std()*np.sqrt(12)
annualized_vol

Lo 20    0.228894
Hi 20    0.172713
dtype: float64

In [37]:
def drawdown(return_series: pd.Series):
    """
    Takes a time series of asset returns
    Computes and returns a DataFrame that contains:
    the wealth index
    the previous peaks
    percent drawdowns
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({
        "Wealth": wealth_index,
        "Peaks": previous_peaks,
        "Drawdown": drawdowns
    })

In [38]:
drawdown(rets["Lo 20"]).head()

,Wealth,Peaks,Drawdown
1926-07,994.300000,994.30000,0.000000
1926-08,1032.481120,1032.48112,0.000000
1926-09,1027.525211,1032.48112,-0.004800
1926-10,993.719631,1032.48112,-0.037542
1926-11,988.254173,1032.48112,-0.042836


In [42]:
drawdown(rets["1999":"2015"]["Lo 20"])["Drawdown"].min()

-0.6248092431236988

In [44]:
drawdown(rets["1999":"2015"]["Lo 20"])["Drawdown"].idxmin()

Period('2009-02', 'M')

In [45]:
drawdown(rets["1999":"2015"]["Hi 20"])["Drawdown"].min()

-0.5527349978713653

In [46]:
drawdown(rets["1999":"2015"]["Hi 20"])["Drawdown"].idxmin()

Period('2009-02', 'M')

In [48]:
hfi = erk.get_hfi_returns()

In [49]:
erk.semideviation(hfi["2009":])

Convertible Arbitrage     0.006681
CTA Global                0.010996
Distressed Securities     0.009498
Emerging Markets          0.015276
Equity Market Neutral     0.005186
Event Driven              0.010135
Fixed Income Arbitrage    0.003321
Global Macro              0.005075
Long/Short Equity         0.012139
Merger Arbitrage          0.003611
Relative Value            0.005711
Short Selling             0.021240
Funds Of Funds            0.007948
dtype: float64

In [52]:
erk.skewness(hfi["2009":]).sort_values()

Equity Market Neutral    -0.896327
Funds Of Funds           -0.646908
Merger Arbitrage         -0.551065
Event Driven             -0.488821
Long/Short Equity        -0.463703
Distressed Securities    -0.254944
Emerging Markets          0.033123
CTA Global                0.052062
Relative Value            0.159953
Global Macro              0.348184
Short Selling             0.456518
Fixed Income Arbitrage    1.121453
Convertible Arbitrage     1.305911
dtype: float64

In [53]:
erk.kurtosis(hfi["2009":]).sort_values()

CTA Global                2.591087
Merger Arbitrage          2.715238
Global Macro              3.164362
Distressed Securities     3.319725
Event Driven              3.620617
Funds Of Funds            3.816132
Long/Short Equity         4.115713
Short Selling             4.175832
Emerging Markets          4.401636
Relative Value            4.512482
Equity Market Neutral     5.071677
Fixed Income Arbitrage    6.406941
Convertible Arbitrage     6.775731
dtype: float64